In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.master", "local[*]") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
    .getOrCreate()


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1139)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1125)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:577)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1666)
	at org.apache.spark.SparkContext.$anonfun$new$13(SparkContext.scala:514)
	at org.apache.spark.SparkContext.$anonfun$new$13$adapted(SparkContext.scala:514)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:514)
	at org.apache.spark.api.java.JavaSparkC

In [ ]:

from pyspark.sql.functions import rand, col, array, concat_ws, lit, monotonically_increasing_id

num_rows = 1000
first_names = ["John", "Jane", "Alice", "Bob", "Charlie", "David", "Emily", "Frank", "Grace", "Henry"]
last_names = ["Smith", "Johnson", "Williams", "Jones", "Brown", "Davis", "Miller", "Wilson", "Moore", "Taylor"]

df = spark.range(num_rows)\
  .withColumn("client_id", monotonically_increasing_id())\
  .withColumn("first_name", array(*[lit(name) for name in first_names]).getItem((rand()*10).cast("int")))\
  .withColumn("last_name", array(*[lit(name) for name in last_names]).getItem((rand()*10).cast("int")))\
  .withColumn("full_name", concat_ws(" ", col("last_name"), col("first_name")))\
  .select(col("client_id"),col("full_name"))

df.show(10)

c:\Users\MattM\anaconda3\lib\site-packages\pyspark\sql\column.py:458: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


+---------+-------------+
|client_id|    full_name|
+---------+-------------+
|        0|Williams John|
|        1| Wilson Frank|
|        2|    Moore Bob|
|        3| Wilson David|
|        4| Wilson Frank|
|        5|  Davis David|
|        6|  Wilson John|
|        7|  Brown Frank|
|        8|  Smith Emily|
|        9| Wilson Grace|
+---------+-------------+
only showing top 10 rows



In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Tokenizer, CountVectorizer, SQLTransformer, RegexTokenizer, NGram, HashingTF
from pyspark.ml.feature import Normalizer, VectorAssembler
from pyspark.ml import Pipeline

from pyspark.sql.functions import col, concat_ws, split, array_sort, min
from pyspark.sql.window import Window
from pyspark.ml.feature import MinHashLSH

from graphframes import *
#parameters
distance_threshold = 0.1

#black box

transform0  =    SQLTransformer(statement=" SELECT * FROM __THIS__" )
token0      =    Tokenizer(inputCol="full_name", outputCol="token" )
transform1  =    SQLTransformer(statement="SELECT *, concat_ws(' ', token) concat FROM __THIS__")
token1      =    RegexTokenizer(pattern="", inputCol="concat", outputCol="char", minTokenLength=1 )
ngram       =    NGram(n=2, inputCol="char", outputCol="ngram")
hasher      =    HashingTF(inputCol="ngram", outputCol="vector")

# add to pipeline when using big data
feat        =    VectorAssembler(inputCols=["vector"], outputCol="features")
kmeans      =    KMeans(k = 2, seed = 1, predictionCol="kmeans")

stages = [transform0,token0,transform1,token1,ngram,hasher] #feat,kmeans

#pre-processing
pipeline = Pipeline(stages=stages)
model = pipeline.fit(df)
model_df = model.transform(df)

 
#knn then jaccard distance
lsh_model = MinHashLSH(inputCol="vector", outputCol="lsh", numHashTables=3).fit(model_df)
similarity_df = lsh_model.approxSimilarityJoin(model_df, model_df, distance_threshold, distCol="text_distance").filter("datasetA.client_id != datasetB.client_id")

 
#graphx
edges = (similarity_df.selectExpr("datasetA.client_id as src","datasetB.client_id as dst")
        .withColumn('set_col', concat_ws(',', col('src'), col('dst')))
        .withColumn('sorted_set', array_sort(split(col('set_col'), ',')))
        .dropDuplicates(['sorted_set']).select(col("src"), col("dst")))

vertices = (similarity_df.selectExpr("datasetA.client_id as id").union(similarity_df.selectExpr("datasetB.client_id as id"))).distinct()


#connections graph
graph_frame = GraphFrame(vertices, edges)

#slow
components_df = graph_frame.connectedComponents().withColumn("min_id", min(col("id")).over(Window.partitionBy("component")))

c:\Users\MattM\anaconda3\lib\site-packages\pyspark\sql\dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling o232.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.net.URLClassLoader.findClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnec

In [ ]:
pip install graphframes

Note: you may need to restart the kernel to use updated packages.


In [ ]:
similarity_df.display()

+--------------------+--------------------+-------------+
|            datasetA|            datasetB|text_distance|
+--------------------+--------------------+-------------+
|{29, Smith David,...|{42949672994, Smi...|          0.0|
|{41, Moore Charli...|{42949673019, Moo...|          0.0|
|{1, Smith Charlie...|{17179869202, Smi...|          0.0|
|{1, Smith Charlie...|{8589934626, Smit...|          0.0|
|{69, Davis Emily,...|{51539607619, Dav...|          0.0|
|{69, Davis Emily,...|{17179869255, Dav...|          0.0|
|{5, Williams Jane...|{34359738388, Wil...|          0.0|
|{10, Taylor David...|{34359738368, Tay...|          0.0|
|{11, Davis Emily,...|{94489280518, Dav...|          0.0|
|{13, Smith Alice,...|{51539607566, Smi...|          0.0|
+--------------------+--------------------+-------------+
only showing top 10 rows



In [ ]:
similarity_df = lsh_model.approxSimilarityJoin(model_df, model_df, distance_threshold, distCol="text_distance").filter("datasetA.client_id != datasetB.client_id")


In [ ]:


scala_version = spark._jvm.scala.util.Properties.versionString()
print(scala_version)
print(spark.version)

version 2.12.17
3.4.0
